In [1]:
import dgl
import os
import torch

def one_hot_embedding(labels, num_classes=11):
    """Embedding labels to one-hot form.

    Args:
      labels: (LongTensor) class labels, sized [N,].
      num_classes: (int) number of classes.

    Returns:
      (tensor) encoded labels, sized [N, #classes].
    """
    y = torch.eye(num_classes)
    # print(" label is",labels)
    return y[labels]

"""
Intent of this function is to convert a DGL-Heterograph into a list of nodes, edges, and edge features per the following format. 
This allows heterohouseganpp to then convert the graph data into a floor plan img.

    "nds": all graph nodes with features in an (Nx13) list. Each node is represented as a one-hot encoded vector with 11 classes, concatinated with the node features [length, door/no door], this is [-1,-1] for room nodes.
    "eds": all graph edges in a Ex3 list, with each edge represented as [src_node_id, +1/-1, dest_node_id] where +1 indicates an edge is present, -1 otherwise
    "eds_f": all graph edge types & features in an Nx3 list, with each entry represented as [edge type (0 - CE, or 1 - RA), edge feature 1, edge feature 2]

Steps:

    1) For nds:
        a) extract node type integers and node features for the exterior walls
        b) create [-1, -1] node features for each of the room-type nodes
        c) one-hot encode the node types
        d) concatenate the features
        e) append to list
    2) For eds / eds_f:
        a) create boilerplate edge list
        b) need to create a mapping from dgl node to graph_list node id
        c) iterate through edges in dgl graph:
          i) switch the correct TWO edges from -1 to 1 based on src/dest node ID from dgl edge
"""

os.chdir("/home/evalexii/Documents/IAAIP/housing-design/housingpipeline/housingpipeline/floor_plan_pipeline/")



In [3]:
g = dgl.load_graphs("./misc/sample_graph.bin")[0][0]
print(type(g))

hg = dgl.to_homogeneous(g, edata=['e'], store_type=True, return_count=True)

# Uncomment to print out partial graph
for c_et in g.canonical_etypes:
    if g.num_edges(c_et) > 0:
        print(f"Edge numbers: {c_et} : {g.num_edges(c_et)}")
        print(f"Edge features: {c_et} :\n {g.edges[c_et].data['e']}")
for nt in g.ntypes:
    if g.num_nodes(nt) > 0:
        print(f"Node features: {nt} :\n {g.nodes[nt].data}")

<class 'dgl.heterograph.DGLGraph'>
Edge numbers: ('bathroom', 'room_adjacency_edge', 'bedroom') : 1
Edge features: ('bathroom', 'room_adjacency_edge', 'bedroom') :
 tensor([[1., 4.]])
Edge numbers: ('bathroom', 'room_adjacency_edge', 'exterior_wall') : 2
Edge features: ('bathroom', 'room_adjacency_edge', 'exterior_wall') :
 tensor([[1., 2.],
        [1., 0.]])
Edge numbers: ('bathroom', 'room_adjacency_edge', 'living_room') : 1
Edge features: ('bathroom', 'room_adjacency_edge', 'living_room') :
 tensor([[0., 5.]])
Edge numbers: ('bedroom', 'room_adjacency_edge', 'bathroom') : 1
Edge features: ('bedroom', 'room_adjacency_edge', 'bathroom') :
 tensor([[1., 0.]])
Edge numbers: ('bedroom', 'room_adjacency_edge', 'exterior_wall') : 2
Edge features: ('bedroom', 'room_adjacency_edge', 'exterior_wall') :
 tensor([[1., 4.],
        [1., 2.]])
Edge numbers: ('bedroom', 'room_adjacency_edge', 'living_room') : 1
Edge features: ('bedroom', 'room_adjacency_edge', 'living_room') :
 tensor([[0., 7.]])

In [5]:
nds = []
eds = []
eds_f = []

names_node_types = g.ntypes
names_canon_etypes = g.canonical_etypes
ids_node_types = hg[0].ndata['_TYPE']
ew_features = g.nodes['exterior_wall'].data['hf']

node_type_counts = hg[1]
num_nodes = torch.sum(torch.tensor(node_type_counts)).item()
canon_etype_counts = hg[2]
num_edges = torch.sum(torch.tensor(canon_etype_counts)).item()

'''
Create node feature tensor
'''
# create empty node feature tensor
nds_f = torch.full((ids_node_types.shape[0],2), fill_value = -1.0)
# find nodetype id of EW's
ew_type_id = names_node_types.index('exterior_wall')
# find node ids of EW's
ew_ids = torch.argwhere(ids_node_types == ew_type_id).flatten()
# insert ew nd features
nds_f[ew_ids] = ew_features

'''
Create one-hot'd node type tensor
'''
nds_t = one_hot_embedding(labels=ids_node_types, num_classes=len(g.ntypes))


'''
Create final nds list
'''
nds = torch.concatenate([nds_t, nds_f], dim=1).tolist()
for nd in nds:
    print(nd)

[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1.0, -1.0]
[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1.0, -1.0]
[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1.0, -1.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 2.0, 1.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, -1.0, -1.0]


In [89]:
'''
Create initial edge tensor with all edges -1
'''

eds = torch.full((num_edges, 3), fill_value= -1)

In [91]:
num_nodes

11

In [76]:
hg[0].ndata['_TYPE'].dim()

1